In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import interplevel, getvar, destagger
from scipy.interpolate import griddata
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### COAMPS settings
nx_c = 301
ny_c = 201
nz_c = 45
coamps_dir = '/p/work1/lloveras/real/nov2018/coamps_files/'

### Latitude and longitude
lats_c = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons_c = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats_c = np.reshape(lats_c, [ny_c, nx_c])
lons_c = np.reshape(lons_c, [ny_c, nx_c])
lons_c[lons_c > 0] -= 360

### Full pressure in hPa
p_c = (np.fromfile(coamps_dir + '9x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
      + np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4'))/2
p_c = np.flip(np.reshape(p_c, [nz_c, ny_c, nx_c]), axis=0)

### Theta
t_c = np.fromfile(coamps_dir + '9x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
t_c = np.flip(np.reshape(t_c, [nz_c, ny_c, nx_c]),axis=0)

### Wind
u_c = np.fromfile(coamps_dir + '9x/uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
v_c = np.fromfile(coamps_dir + '9x/vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
w_c = np.fromfile(coamps_dir + '9x/wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
u_c = np.flip(np.reshape(u_c, [nz_c, ny_c, nx_c]),axis=0)
v_c = np.flip(np.reshape(v_c, [nz_c, ny_c, nx_c]),axis=0)
w_c = np.flip(np.reshape(w_c, [nz_c+1, ny_c, nx_c]),axis=0)
w_c_ds = destagger(w_c,stagger_dim=0)

### Water vapor
qv_c = np.fromfile(coamps_dir + '9x/wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
qv_c = np.flip(np.reshape(qv_c, [nz_c, ny_c, nx_c]),axis=0)


In [ ]:
### Plot COAMPS perturbations
lev = 850
dt500 = np.nan_to_num(interplevel(t_c, p_c, lev))
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/2.

# Create a figure
fig = plt.figure(figsize=(10,6),dpi=200)
ax = plt.axes(projection=crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70)))

# Plot the data
plt.contourf(lons_c, lats_c, dt500, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())
plt.colorbar()

states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces")

ax.add_feature(states, linewidth=.5, edgecolor="grey")
ax.coastlines('50m', linewidth=0.8,color='grey')

gls = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False

### Plot target box
min_lat = 25
max_lat = 40
min_lon = -107
max_lon = -92

# ax.plot([min_lon, min_lon, max_lon, max_lon, min_lon], [min_lat, max_lat, max_lat, min_lat, min_lat], 
#          color='k', linewidth=1, marker='.',
#          transform=crs.PlateCarree())

plt.show()


In [ ]:
### WRF 30-km settings
nx_w = 400
ny_w = 250
nz_w = 44
ctl_dir = '/p/work1/lloveras/real/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00'

### Open the netCDF file for reading
ncfile_ctl = Dataset(ctl_dir)

### Latitude and longitude
lats_w = np.asarray(getvar(ncfile_ctl,'lat',timeidx=0))
lons_w = np.asarray(getvar(ncfile_ctl,'lon',timeidx=0))
lons_w[lons_w > 0] -= 360

### Full pressure in hPa
p_w = np.asarray(getvar(ncfile_ctl,'pressure',timeidx=0))

### Heights in m
z_w = np.asarray(getvar(ncfile_ctl,'z',timeidx=0))


In [ ]:
### Interpolate perturbations onto WRF grid

### Start by interpolating WRF pressure field onto COAMPS horizontal grid
p_w_cxy = np.zeros((nz_w,ny_c,nx_c))
for k in range(nz_w):
    p_w_cxy[k,:,:] = griddata((lons_w.ravel(), lats_w.ravel()),
                              p_w[k,:,:].ravel(), (lons_c, lats_c), method='linear')

### Now interpolate COAMPS perturbations onto WRF pressure levels
t_c_wz = np.zeros([nz_w, ny_c, nx_c])
u_c_wz = np.zeros((nz_w, ny_c, nx_c))
v_c_wz = np.zeros((nz_w, ny_c, nx_c))
w_c_wz = np.zeros((nz_w, ny_c, nx_c))
qv_c_wz = np.zeros((nz_w, ny_c, nx_c))
for k in range(nz_w):
    t_c_wz[k,:,:] = np.nan_to_num(interplevel(t_c, p_c, p_w_cxy[k,:,:]))
    u_c_wz[k,:,:] = np.nan_to_num(interplevel(u_c, p_c, p_w_cxy[k,:,:]))
    v_c_wz[k,:,:] = np.nan_to_num(interplevel(v_c, p_c, p_w_cxy[k,:,:]))
    w_c_wz[k,:,:] = np.nan_to_num(interplevel(w_c_ds, p_c, p_w_cxy[k,:,:]))
    qv_c_wz[k,:,:] = np.nan_to_num(interplevel(qv_c, p_c, p_w_cxy[k,:,:]))

### Finally interpolate perturbations onto WRF horizontal grid
t_c_w = np.zeros((nz_w, ny_w, nx_w))
u_c_w = np.zeros((nz_w, ny_w, nx_w))
v_c_w = np.zeros((nz_w, ny_w, nx_w))
w_c_w = np.zeros((nz_w, ny_w, nx_w))
qv_c_w = np.zeros((nz_w, ny_w, nx_w))
for k in range(nz_w):
    t_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          t_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    u_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          u_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    v_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          v_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    w_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          w_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    qv_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          qv_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))


In [ ]:
### Plot WRF perturbations
lev = 950.
dt500 = np.nan_to_num(interplevel(t_c_w, p_w, lev))
z500 = interplevel(z_w, p_w, lev)
zlvls = np.arange(0,1500,8)
thlvls = np.arange(-2,2.25,0.25)

# Create a figure
fig = plt.figure(figsize=(10,6),dpi=200)
ax = plt.axes(projection=crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70)))

# Plot the data

cs = ax.contour(lons_w, lats_w, z500/10.,levels=zlvls,
                colors='k',alpha=0.5,
                transform=crs.PlateCarree())
ax.clabel(cs,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

plt.contourf(lons_w, lats_w, dt500, levels=thlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())
plt.colorbar()

states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces")

ax.add_feature(states, linewidth=.5, edgecolor="grey")
ax.coastlines('50m', linewidth=0.8,color='grey')

gls = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False

### Plot target box
# min_lat = 25
# max_lat = 40
# min_lon = -107
# max_lon = -92
min_lat = 17
max_lat = 32
min_lon = -96
max_lon = -81

ax.plot([min_lon, min_lon, max_lon, max_lon, min_lon], [min_lat, max_lat, max_lat, min_lat, min_lat], 
         color='k', linewidth=1, marker='.',
         transform=crs.PlateCarree())

plt.show()


In [ ]:
### Rescale perturbations to retain magnitude prior to interpolation

rescale_unity = False

t_max_c = np.amax(np.abs(t_c))
u_max_c = np.amax(np.abs(u_c))
v_max_c = np.amax(np.abs(v_c))
w_max_c = np.amax(np.abs(w_c))
qv_max_c = np.amax(np.abs(qv_c))
 
t_max_w = np.amax(np.abs(t_c_w))
u_max_w = np.amax(np.abs(u_c_w))
v_max_w = np.amax(np.abs(v_c_w))
w_max_w = np.amax(np.abs(w_c_w))
qv_max_w = np.amax(np.abs(qv_c_w))    
    
t_r = t_max_c/t_max_w
u_r = u_max_c/u_max_w
v_r = v_max_c/v_max_w
w_r = w_max_c/w_max_w
qv_r = qv_max_c/qv_max_w

t_full = t_c_w*t_r
u_full = u_c_w*u_r
v_full = v_c_w*v_r
w_full = w_c_w*w_r
qv_full = qv_c_w*qv_r

if rescale_unity:
    s = max(t_max_c,u_max_c,qv_max_c*1000.)
    t_full = t_full/s
    u_full = u_full/s
    v_full = v_full/s
    w_full = w_full/s
    qv_full = qv_full/s


In [ ]:
### Filtering

lam = 1000.
dx = 30.
kx = np.fft.fftfreq(nx_w,dx)
ky = np.fft.fftfreq(ny_w,dx)
peak = 1/lam
large_scale = False
small_scale = True

### Large scale

if large_scale:

    t_fft = np.fft.fft2(t_full)
    t_fft[:,np.abs(ky) > peak, :] = 0.
    t_fft[:,:,np.abs(kx) > peak] = 0.
    t_filt = np.real(np.fft.ifft2(t_fft))

    u_fft = np.fft.fft2(u_full)
    u_fft[:,np.abs(ky) > peak, :] = 0.
    u_fft[:,:,np.abs(kx) > peak] = 0.
    u_filt = np.real(np.fft.ifft2(u_fft))

    v_fft = np.fft.fft2(v_full)
    v_fft[:,np.abs(ky) > peak, :] = 0.
    v_fft[:,:,np.abs(kx) > peak] = 0.
    v_filt = np.real(np.fft.ifft2(v_fft))

    w_fft = np.fft.fft2(w_full)
    w_fft[:,np.abs(ky) > peak, :] = 0.
    w_fft[:,:,np.abs(kx) > peak] = 0.
    w_filt = np.real(np.fft.ifft2(w_fft))

    qv_fft = np.fft.fft2(qv_full)
    qv_fft[:,np.abs(ky) > peak, :] = 0.
    qv_fft[:,:,np.abs(kx) > peak] = 0.
    qv_filt = np.real(np.fft.ifft2(qv_fft))

### Large scale

if small_scale:

    t_fft = np.fft.fft2(t_full)
    t_fft[:,np.abs(ky) < peak, :] = 0.
    t_fft[:,:,np.abs(kx) < peak] = 0.
    t_filt = np.real(np.fft.ifft2(t_fft))

    u_fft = np.fft.fft2(u_full)
    u_fft[:,np.abs(ky) < peak, :] = 0.
    u_fft[:,:,np.abs(kx) < peak] = 0.
    u_filt = np.real(np.fft.ifft2(u_fft))

    v_fft = np.fft.fft2(v_full)
    v_fft[:,np.abs(ky) < peak, :] = 0.
    v_fft[:,:,np.abs(kx) < peak] = 0.
    v_filt = np.real(np.fft.ifft2(v_fft))

    w_fft = np.fft.fft2(w_full)
    w_fft[:,np.abs(ky) < peak, :] = 0.
    w_fft[:,:,np.abs(kx) < peak] = 0.
    w_filt = np.real(np.fft.ifft2(w_fft))

    qv_fft = np.fft.fft2(qv_full)
    qv_fft[:,np.abs(ky) < peak, :] = 0.
    qv_fft[:,:,np.abs(kx) < peak] = 0.
    qv_filt = np.real(np.fft.ifft2(qv_fft))
    

In [ ]:
### Lat-lon box

# min_lat = 25
# max_lat = 40
# min_lon = -107
# max_lon = -92

min_lat = 17
max_lat = 32
min_lon = -96
max_lon = -81

box = True
hole = False

if box:

    t_trunc = np.copy(t_full)
    u_trunc = np.copy(u_full)
    v_trunc = np.copy(v_full)
    w_trunc = np.copy(w_full)
    qv_trunc = np.copy(qv_full)

    t_trunc[:,lons_w > max_lon] = np.NaN
    t_trunc[:,lons_w < min_lon] = np.NaN
    t_trunc[:,lats_w > max_lat] = np.NaN
    t_trunc[:,lats_w < min_lat] = np.NaN

    u_trunc[:,lons_w > max_lon] = np.NaN
    u_trunc[:,lons_w < min_lon] = np.NaN
    u_trunc[:,lats_w > max_lat] = np.NaN
    u_trunc[:,lats_w < min_lat] = np.NaN

    v_trunc[:,lons_w > max_lon] = np.NaN
    v_trunc[:,lons_w < min_lon] = np.NaN
    v_trunc[:,lats_w > max_lat] = np.NaN
    v_trunc[:,lats_w < min_lat] = np.NaN

    w_trunc[:,lons_w > max_lon] = np.NaN
    w_trunc[:,lons_w < min_lon] = np.NaN
    w_trunc[:,lats_w > max_lat] = np.NaN
    w_trunc[:,lats_w < min_lat] = np.NaN

    qv_trunc[:,lons_w > max_lon] = np.NaN
    qv_trunc[:,lons_w < min_lon] = np.NaN
    qv_trunc[:,lats_w > max_lat] = np.NaN
    qv_trunc[:,lats_w < min_lat] = np.NaN

    t_trunc = np.nan_to_num(t_trunc)
    u_trunc = np.nan_to_num(u_trunc)
    v_trunc = np.nan_to_num(v_trunc)
    w_trunc = np.nan_to_num(w_trunc)
    qv_trunc = np.nan_to_num(qv_trunc)

if hole:

    t_trunc = np.copy(t_full)
    u_trunc = np.copy(u_full)
    v_trunc = np.copy(v_full)
    w_trunc = np.copy(w_full)
    qv_trunc = np.copy(qv_full)

    t_trunc[:,(lons_w > min_lon) & (lons_w < max_lon)
               & (lats_w > min_lat) & (lats_w < max_lat)] = np.NaN
    
    u_trunc[:,(lons_w > min_lon) & (lons_w < max_lon)
               & (lats_w > min_lat) & (lats_w < max_lat)] = np.NaN
    
    v_trunc[:,(lons_w > min_lon) & (lons_w < max_lon)
               & (lats_w > min_lat) & (lats_w < max_lat)] = np.NaN
    
    w_trunc[:,(lons_w > min_lon) & (lons_w < max_lon)
               & (lats_w > min_lat) & (lats_w < max_lat)] = np.NaN
    
    qv_trunc[:,(lons_w > min_lon) & (lons_w < max_lon)
               & (lats_w > min_lat) & (lats_w < max_lat)] = np.NaN

    t_trunc = np.nan_to_num(t_trunc)
    u_trunc = np.nan_to_num(u_trunc)
    v_trunc = np.nan_to_num(v_trunc)
    w_trunc = np.nan_to_num(w_trunc)
    qv_trunc = np.nan_to_num(qv_trunc)
    

In [ ]:
### Rescale to have equal DTE

dte_full = np.sum(u_full**2) + np.sum(v_full**2) + (CP/TR)*np.sum(t_full**2)
dte_filt = np.sum(u_filt**2) + np.sum(v_filt**2) + (CP/TR)*np.sum(t_filt**2)
dte_trunc = np.sum(u_trunc**2) + np.sum(v_trunc**2) + (CP/TR)*np.sum(t_trunc**2)

r_filt = np.sqrt(dte_full/dte_filt)
t_filt = t_filt*r_filt
u_filt = u_filt*r_filt
v_filt = v_filt*r_filt
w_filt = w_filt*r_filt
qv_filt = qv_filt*r_filt

r_trunc = np.sqrt(dte_full/dte_trunc)
t_trunc = t_trunc*r_trunc
u_trunc = u_trunc*r_trunc
v_trunc = v_trunc*r_trunc
w_trunc = w_trunc*r_trunc
qv_trunc = qv_trunc*r_trunc


In [ ]:
### Perturbation file settings: 30km
pert_dir = '/p/work1/lloveras/real/nov2018/30km_files/adj_cape_box/wrfin_d01_2018-11-13_12_00_00'

### Open the netCDF file for writing
ncfile_pert = Dataset(pert_dir,'r+')

### Theta
t = np.asarray(ncfile_pert.variables['THM'][0,:,:,:])
ncfile_pert.variables['THM'][0,:,:,:] = t + t_trunc

### Wind
u = np.asarray(ncfile_pert.variables['U'][0,:,:,:])
ncfile_pert.variables['U'][0,:,:,:-1] = u[:,:,:-1] + u_trunc
ncfile_pert.variables['U'][0,:,:,-1] = u[:,:,-1]

v = np.asarray(ncfile_pert.variables['V'][0,:,:,:])
ncfile_pert.variables['V'][0,:,:-1,:] = v[:,:-1,:] + v_trunc
ncfile_pert.variables['V'][0,:,-1,:] = v[:,-1,:]

w = np.asarray(ncfile_pert.variables['W'][0,:,:,:])
ncfile_pert.variables['W'][0,:-1,:,:] = w[:-1,:,:] + w_trunc
ncfile_pert.variables['W'][0,-1,:,:] = w[-1,:,:]

### Water vapor
qv = np.asarray(ncfile_pert.variables['QVAPOR'][0,:,:,:])
ncfile_pert.variables['QVAPOR'][0,:,:,:] = qv + qv_trunc

### Close the perturbed netCDF file
ncfile_pert.close()
